This lesson is better to run on environment powerful computing resources, I would suggest with 4090(24GB) or higher rank. 

Otherwise like mine 4060TI(16GB) might not be good to run qwen-2.5-coder model with somalagent. 

the reason is the agent framework does support quantization with 16bit. 
Check this github issue for more information. 

https://github.com/huggingface/smolagents/issues/969

You can see mine is using similar configuration but I have to use 1.5B if I would want to get a result. however, the quality of codes from 1.5B is not good enough. It is still not good with 3B and ran out of memory with 7B.

In [1]:
!pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q

In [2]:
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # 货运飞机的平均速度
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # 提取坐标
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # 地球半径（公里）
    EARTH_RADIUS_KM = 6371.0

    # 使用半正矢公式计算大圆距离
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # 增加10%以考虑非直接路线和空中交通管制
    actual_distance = distance * 1.1

    # 计算飞行时间
    # 为起飞和着陆程序增加1小时
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # 格式化结果
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


In [3]:
from PIL import Image
from smolagents import CodeAgent, DuckDuckGoSearchTool, TransformersModel, VisitWebpageTool

In [4]:
import logging

logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger('ModelLoader')

In [5]:
import torch
import accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes as bnb

from transformers import TRANSFORMERS_CACHE

In [ ]:
# Print versions and CUDA availability
print(f"PyTorch version: {torch.__version__}")
print(f"Accelerate version: {accelerate.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Current CUDA device: {torch.cuda.current_device()}")
print(f"CUDA device name: {torch.cuda.get_device_name(0)}")

In [30]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_in_8bit=False,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

# Use a smaller model to reduce memory requirements
model_id = 'Qwen/Qwen2.5-Coder-3B-Instruct'
logger.info(f"Loading model {model_id}...")

# Try to clear CUDA cache before loading model
torch.cuda.empty_cache()

2025-03-17 10:42:12,008 - ModelLoader - INFO - Loading model Qwen/Qwen2.5-Coder-3B-Instruct...


In [31]:
try:
    # Let TransformersModel handle model loading directly
    model = TransformersModel(
        model_id=model_id,
        device_map="auto",
        #torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        max_new_tokens=1024,
        kwargs={
            "quantization_config": quantization_config,
            "low_cpu_mem_usage": True
        }
    )
    
    logger.info("Model loaded successfully!")
    
except Exception as e:
    logger.error(f"Error loading model: {e}")
    raise

2025-03-17 10:42:13,701 - smolagents.models - INFO - Using device: auto
2025-03-17 10:42:14,082 - accelerate.utils.modeling - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2025-03-17 10:42:19,387 - accelerate.big_modeling - WARNING - Some parameters are on the meta device because they were offloaded to the cpu.
2025-03-17 10:42:19,925 - ModelLoader - INFO - Model loaded successfully!


In [32]:
search_tool = DuckDuckGoSearchTool()

In [33]:
agent = CodeAgent(
    model=model,
    tools=[search_tool, VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [34]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [ ]:
result = agent.run(task)
print(f"Result: {result}")

In [35]:
agent.planning_interval = 4
# analysis from the code above"
	# •	The agent follows a step-by-step approach to complete the task.
	# •	It has a maximum of 20 steps (max_steps=20).
	# •	Every 4 steps, it pauses to reassess its current plan before continuing.
	# •	This allows the agent to adjust its strategy dynamically instead of rigidly following an initial plan.

# detailed_report = agent.run(f"""
# You're an expert analyst. You make comprehensive reports after visiting many websites.
# Don't hesitate to search for many queries at once in a for loop.
# For each data point that you find, visit the source url to confirm numbers.

# {task}
# """)

# the below did not work well
# detailed_report = agent.run(f"""
# You're an expert analyst. You make concise and accurate reports.
# Search for information in a structured manner. Limit your queries to a reasonable number.
# Visit a maximum of 10 high-quality sources to verify the numbers.

# {task}
# """)

#### not working well

# detailed_report = agent.run(f"""
# You're an expert analyst. You make comprehensive reports after visiting websites.
# To ensure accuracy, search for at most 5 queries at a time before analyzing results.
# Prioritize high-quality sources, and visit the source URL to confirm numbers.

# {task}
# """)

# my environment is 12600KF + 32GB + 4060TI16GB

detailed_report = agent.run(f"""
You're an expert analyst. Your goal is to generate a **concise** and **efficient** report.
- Execute a **maximum of 3 search queries in total**.
- Visit **no more than 5 web pages** to verify numbers.
- Avoid redundant searches and prioritize **high-quality, reliable sources**.
- Summarize findings in a structured manner **without unnecessary details**.
- Keep the response **short (under 500 words) and to the point**.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. Your goal is to generate a **concise** and **efficient** report.                      │
│ - Execute a **maximum of 3 search queries in total**.                                                           │
│ - Visit **no more than 5 web pages** to verify numbers.                                                         │
│ - Avoid redundant searches and prioritize **high-quality, reliable sources**.                                   │
│ - Summarize findings in a structured manner **without unnecessary details**.                                    │
│ - Keep the response **short (under 500 words) and to the point**.                                               │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ TransformersModel - Qwen/Qwen2.5-Coder-3B-Instruct ────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
```


```plaintext
1. Search for "Batman filming locations" using DuckDuckGo.
2. Identify the top 3 relevant Wikipedia articles from the search results.
3. Extract the locations from these articles.
4. Search for "Gotham City flight data" using DuckDuckGo.
5. Find the closest Batman filming location to Gotham City.
6. Calculate the distance between Gotham City and the closest Batman filming location.
7. Calculate the flight duration from Gotham City to the closest Batman filming location using the calculated 
distance and average cargo plane speed (750 km/h).
8. Search for "supercar factory locations" using DuckDuckGo.
9. Find the supercar factories with the same cargo plane transfer time as the calculated flight duration.
10. Compile the results into a pandas dataframe.
11. Format the dataframe to include columns for the Batman filming location, distance, flight duration, and 
supercar factory.
12. Present the final answer in a concise and efficient manner.
<end_plan>
```
```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
                                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out: None

...

Out: None

[Step 7: Duration 8.06 seconds| Input tokens: 34,926 | Output tokens: 342]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 8: Duration 105.06 seconds| Input tokens: 41,501 | Output tokens: 347]

KeyboardInterrupt: 

In [12]:
quantization_config = BitsAndBytesConfig(
    load_in_8bit=False,
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # 使用 FP16 加速计算
    bnb_4bit_use_double_quant=True        # 启用双重量化以减少显存占用
)

In [ ]:
print(f'Quntization Config: {quantization_config}')
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
print("Loading model...")

max_memory = {
    0: "14GB",  # 主 GPU 使用上限
    "cpu": "26GB"       # CPU 使用上限
}

# Try loading with quantization config
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    offload_buffers=True,
    max_memory = max_memory,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

print(f"Model Config: {model.config}")
print("Model loaded. Starting inference.")